### Scryfall API Setup

In [1]:
SF_API_URL = 'https://api.scryfall.com'
SF_API_HEADERS = {
    'User_Agent' : 'card-classifier/0.1',
    'Accept' : '*/*'
}
SF_API_DELAY = 50 # ms - No ratelimit on *.scryfall.io

In [ ]:
import requests
import json
import os

### Pull all download URIs

response = requests.get(SF_API_URL+'/bulk-data', headers=SF_API_HEADERS)

data = response.json()['data']
# print(json.dumps(data, indent=4))

# Extract all URI
download_uri = {}
for object in data:
    download_uri[object['type']] = object['download_uri']
print("URI paths saved")

# Download and save the unique artwork file
artwork_file_path = os.path.join('data', 'unique_artwork.json')
response = requests.get(download_uri['unique_artwork'], headers=SF_API_HEADERS)
with open(artwork_file_path, 'wb') as file:
    file.write(response.content)

URI paths saved


### Download Card Images

In [3]:
with open(artwork_file_path, 'r', encoding="utf8") as file:
    unique_artwork_data = json.load(file)

In [ ]:
# print(json.dumps(unique_artwork_data[4], indent=4))

In [ ]:
from concurrent.futures import ThreadPoolExecutor
# from tqdm import tqdm

def download_image(entry, is_dfc=False, card_face=None):
    if not is_dfc:
        filename = f"{entry['name'].replace('/', '_')}_{entry['set']}_{entry['collector_number']}_{entry['id']}.jpg"
        art_path = os.path.join('data/art', filename)
        image_url = entry['image_uris']['art_crop']
    else:
        filename = f"{card_face['name'].replace('/', '_')}_{entry['set']}_{entry['collector_number']}_{entry['id']}.jpg"
        art_path = os.path.join('data/art', filename)
        image_url = card_face['image_uris']['art_crop']

    response = requests.get(image_url, headers=SF_API_HEADERS)
    if response.status_code == 200:
        with open(art_path, 'wb') as file:
            file.write(response.content)
        # print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {filename} (Status code: {response.status_code})")

with ThreadPoolExecutor() as executor:
    futures = []
    for art_entry in unique_artwork_data:
        # If not DFC
        if 'image_uris' in art_entry:
            future = executor.submit(download_image, art_entry)
            futures.append(future)
        # If DFC
        else:
            for card_face in art_entry['card_faces']:
                future = executor.submit(download_image, art_entry, True, card_face)
                futures.append(future)


### Identify Already Tagged Cards

In [12]:
search_query = 'arttag:bisexual-lighting'
# search_query = 'odric'
search_unique = 'art'
search_include_extras = 'true'

# If result length is 175, likely another page
data_len = 175
search_data = []
page = 1

while data_len == 175:
    response = requests.get(
        SF_API_URL + '/cards/search',
        headers=SF_API_HEADERS,
        params={
            'q': search_query,
            'unique': search_unique,
            'include_extras': search_include_extras,
            'page': page
        }
    )
    search_data.extend(response.json()['data'])
    data_len = len(response.json()['data'])
    page += 1
    print(page)
    print(len(response.json()['data']))

2
175
3
116


In [13]:
len(search_data)

291

In [24]:
art_list = os.listdir('data/art')

In [45]:
len(art_list)

45385

In [44]:
art_list = os.listdir('data/art')


search_art = {card['name']: {'id': card['id'], 'path': path} for card in search_data for path in art_list if path.split('_')[-1][:-4] == card['id']}

In [42]:
search_art

{'Absorb Energy': {'id': 'bfdca67d-9a97-4ddc-8d50-26a48ad2e4b7',
  'path': 'Absorb Energy_ymid_12_bfdca67d-9a97-4ddc-8d50-26a48ad2e4b7.jpg'},
 'Aegar, the Freezing Flame': {'id': '0a0d2ebd-99ba-4968-a79e-68288626bed1',
  'path': 'Aegar, the Freezing Flame_mul_31_0a0d2ebd-99ba-4968-a79e-68288626bed1.jpg'},
 'Aeon Engine': {'id': 'ea4dfdd7-f35a-45d6-9fc7-2c096f92dd5f',
  'path': 'Aeon Engine_c19_52_ea4dfdd7-f35a-45d6-9fc7-2c096f92dd5f.jpg'},
 'Aerial Boost': {'id': 'f7017afb-4c7c-4c8d-9c9d-3f056a55561e',
  'path': 'Aerial Boost_mom_2_f7017afb-4c7c-4c8d-9c9d-3f056a55561e.jpg'},
 'Alabaster Host Intercessor': {'id': '165357cc-ec74-490f-aec3-7048bb43c8f9',
  'path': 'Alabaster Host Intercessor_mom_3_165357cc-ec74-490f-aec3-7048bb43c8f9.jpg'},
 'All-Seeing Arbiter': {'id': '0ac7e199-c32e-4479-a92f-df52f1182c9a',
  'path': 'All-Seeing Arbiter_snc_286_0ac7e199-c32e-4479-a92f-df52f1182c9a.jpg'},
 'Archaeomancer': {'id': '552bb59c-085f-4caf-8430-ba6e2888ea6d',
  'path': 'Archaeomancer_sld_283_55

In [43]:
len(search_art)

272